Sistema mas eficiente

In [1]:
import pandas as pd
import numpy as np

In [3]:
df=pd.read_csv(r'D:\OneDrive\Escritorio\Proyecto 1\Proyecto-Individual-1-ML\data procesada para funciones\data_ml')
df=df[['Id_Pelicula','title', 'coleccion', 'overview', 'genero_1']]
df=df.fillna('')
df=df.iloc[:25000,:]
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Id_Pelicula  25000 non-null  object
 1   title        25000 non-null  object
 2   coleccion    25000 non-null  object
 3   overview     25000 non-null  object
 4   genero_1     25000 non-null  object
dtypes: object(5)
memory usage: 976.7+ KB


In [4]:
df['combined_features'] = df['overview'] + ' ' + df['genero_1'] + ' ' + df['coleccion']
df['combined_features']

0        Led by Woody, Andy's toys live happily in his ...
1        When siblings Judy and Peter discover an encha...
2        A family wedding reignites the ancient feud be...
3        Cheated on, mistreated and stepped on, the wom...
4        Just when George Banks has recovered from his ...
                               ...                        
24995    Izzy is raising her younger brother, Kevin, by...
24996    During the holiday season, when the animals of...
24997    A tough detective who is part of an elite New ...
24998    V/H/S: VIRAL's segments include the story of a...
24999    'Running from Crazy' is a documentary examinin...
Name: combined_features, Length: 25000, dtype: object

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')
matriz_tfidf_1 = tfidf.fit_transform(df['combined_features'])

In [6]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(matriz_tfidf_1, matriz_tfidf_1)

In [8]:
indices = pd.Series(df.index, index=df['title']).drop_duplicates()

def get_recomendacion(title, cosine_sim=cosine_sim):
    if title not in indices:
        return "El titulo ingresado no se encuentra en el dataset, por favor vuelva a intentar"
    idx = indices[title]

    sim_scores = list(enumerate(cosine_sim[idx]))

    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    sim_scores = sim_scores[1:6]

    movie_indices = [i[0] for i in sim_scores]

    return df['title'].iloc[movie_indices]

In [10]:
print(get_recomendacion('Star Wars'))

1152                               The Empire Strikes Back
1165                                    Return of the Jedi
15446    Empire of Dreams: The Story of the Star Wars T...
1265                                          Mad Dog Time
309                                      The Swan Princess
Name: title, dtype: object


In [11]:
df.drop(['coleccion','overview','genero_1'],axis=1,inplace=True)
df

,Id_Pelicula,title,combined_features
0,862.0,Toy Story,"Led by Woody, Andy's toys live happily in his ..."
1,8844.0,Jumanji,When siblings Judy and Peter discover an encha...
2,15602.0,Grumpier Old Men,A family wedding reignites the ancient feud be...
3,31357.0,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom..."
4,11862.0,Father of the Bride Part II,Just when George Banks has recovered from his ...
...,...,...,...
24995,23515.0,Sick Girl,"Izzy is raising her younger brother, Kevin, by..."
24996,10258.0,The Madagascar Penguins in a Christmas Caper,"During the holiday season, when the animals of..."
24997,32080.0,The Seven-Ups,A tough detective who is part of an elite New ...
24998,267806.0,V/H/S: Viral,V/H/S: VIRAL's segments include the story of a...


In [12]:
#np.save('cosine_sim.npy', cosine_sim)
#df.to_csv('data_funcion_recomendacion')